In [1]:
import matplotlib
%matplotlib inline

import numpy as np
import time
import cv2

import os
from lava.magma.core.run_configs import Loihi2SimCfg, Loihi2HwCfg
from lava.magma.core.run_conditions import RunSteps, RunContinuous
from lava.magma.core.decorator import implements, requires, tag
from lava.magma.core.model.py.model import PyLoihiProcessModel
from lava.magma.core.model.py.ports import PyOutPort, PyInPort
from lava.magma.core.model.py.type import LavaPyType
from lava.magma.core.process.ports.ports import OutPort, InPort
from lava.magma.core.process.process import AbstractProcess
from lava.magma.core.process.variable import Var
from lava.magma.core.resources import CPU
from lava.magma.core.sync.protocols.loihi_protocol import LoihiProtocol
from lava.proc.io.sink import RingBuffer

from lava.lib.peripherals.dvs.inivation import InivationCamera
from metavision_core.utils import get_sample
from metavision_sdk_cv import TrailFilterAlgorithm, ActivityNoiseFilterAlgorithm
from metavision_sdk_core import PolarityFilterAlgorithm

from multiprocessing import Pipe
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import IPython


from metavision_core.event_io import RawReader
from lava.lib.peripherals.dvs.transform import Compose, MirrorHorizontally, MirrorVertically, MergePolarities, Downsample
from metavision_ml.preprocessing import histo, histo_quantized
from metavision_core.event_io import RawReader

from lava.proc.embedded_io.spike import PyToNxAdapter, NxToPyAdapter
from lava.proc.lif.process import LIF, LogConfig
from lava.proc.dense.process import Dense
from lava.proc.sparse.process import Sparse
import logging
from lava.proc.dense.models import PyDenseModelBitAcc
from lava.proc.lif.models import PyLifModelBitAcc
from lava.proc.sparse.models import PySparseModelBitAcc

from utils import VisProcess, VisSwipeProcess
from scipy.sparse import csr_matrix

/usr/lib/python3/dist-packages/metavision_core_ml/preprocessing/viz.py:41: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _viz_events(events, img):


In [7]:
SEQUENCE_FILENAME_RAW = "traffic_monitoring.raw"
# if the file doesn't exist, it will be downloaded from Prophesee's public sample server
get_sample(SEQUENCE_FILENAME_RAW)

reader = RawReader(SEQUENCE_FILENAME_RAW)
height, width = reader.get_size()
del reader
# width, height = (640, 480)


...99%, 70 MB, 3214 KB/s, 22 seconds passed

## Swipe detector

In the next section, we will create a simple hand gesture recognition model. For this purpose, we create a simple network in Lava in which we hand crafted the connectivity to detect general motion in the input stream.

### Connectivity

There are four population of LIF neurons detecting the directions 'up', 'down', 'left' and 'right'. Each population is recurrently connected with a shifted local kernel. For example the neurons in the up channel connects such that a neuron excites its neurons above. If the movement in the image is indeed upwards, there will be more events in the 'up' population compared to the others. This number of events in each population is used to predict the overall movement in the image.



<table>
<tr>
    <td> UP</td>
    <td> RIGHT </td>
    <td> DOWN </td>
    <td> LEFT </td>
</tr>
<tr>
    <td> <img src="imgs/swipe_kernel_up.png" alt="Drawing" style="height: 250px;"/> </td>
    <td> <img src="imgs/swipe_kernel_right.png" alt="Drawing" style="height: 250px;"/> </td>
    <td> <img src="imgs/swipe_kernel_down.png" alt="Drawing" style="height: 250px;"/> </td>
    <td> <img src="imgs/swipe_kernel_left.png" alt="Drawing" style="height: 250px;"/> </td>
</tr>
</table>

In [8]:

from scipy.sparse import csr_matrix

transformations = Compose(
    [
        Downsample(factor=0.2),
        MergePolarities(),
    ]
)


print("Create Cam")
# Init Processes
# frame_input = InivationCamera(device="",
#                               transformations=transformations,
#                               sensor_shape=(height, width),
#                               num_output_time_bins=1)

frame_input = PropheseeCamera(filename="",
                              transformations=transformations,
                              sensor_shape=(height, width),
                              num_output_time_bins=1)

num_neurons = np.prod(frame_input.s_out.shape)
print("NUM NEURONS", num_neurons)
_, _, scaled_height, scaled_width = frame_input.s_out.shape
flat_shape = (num_neurons, )
num_out_neurons = 200

w_ff = 400
w_rec = 5
w_o = 50
kernel_size = 5

vth = 1000
du = dv = 2000


print("Create FF weights")

ff_weights = csr_matrix(np.eye(num_neurons) * w_ff)


print("Create down weights")
rec_weights_down = np.zeros((num_neurons, num_neurons)).reshape((scaled_height, scaled_width, scaled_height, scaled_width))
for i in range(scaled_height):
    for j in range(scaled_width):
        rec_weights_down[i+1:i+kernel_size+1, j-kernel_size:j+kernel_size, i, j-kernel_size:j+kernel_size] = w_rec
rec_weights_down = csr_matrix(rec_weights_down.reshape((num_neurons, num_neurons)))

print("Create up weights")
rec_weights_up = np.zeros((num_neurons, num_neurons)).reshape((scaled_height, scaled_width, scaled_height, scaled_width))
for i in range(scaled_height):
    for j in range(scaled_width):
        rec_weights_up[i-kernel_size:i, j-kernel_size:j+kernel_size, i, j-kernel_size:j+kernel_size] = w_rec
rec_weights_up = csr_matrix(rec_weights_up.reshape((num_neurons, num_neurons)))

print("Create left weights")
rec_weights_left = np.zeros((num_neurons, num_neurons)).reshape((scaled_height, scaled_width, scaled_height, scaled_width))
for i in range(scaled_height):
    for j in range(scaled_width):
        rec_weights_left[i-kernel_size:i+kernel_size, j-kernel_size:j, i-kernel_size:i+kernel_size, j] = w_rec

rec_weights_left = csr_matrix(rec_weights_left.reshape((num_neurons, num_neurons)))

print("Create right weights")
rec_weights_right = np.zeros((num_neurons, num_neurons)).reshape((scaled_height, scaled_width, scaled_height, scaled_width))
for i in range(scaled_height):
    for j in range(scaled_width):
        rec_weights_right[i-kernel_size:i+kernel_size, j+1:j+kernel_size+1, i-kernel_size:i+kernel_size, j] = w_rec
rec_weights_right = csr_matrix(rec_weights_right.reshape((num_neurons, num_neurons)))

print("Create out weights")
w_out = np.zeros((num_out_neurons, num_neurons))
tmp = np.kron(np.eye(num_out_neurons, dtype=np.int32), np.array([w_o] * (num_neurons // num_out_neurons)))
w_out[:tmp.shape[0], :tmp.shape[1]] = tmp
w_out = csr_matrix(w_out)

Create Cam
NUM NEURONS 12288
Create FF weights
Create down weights
Create up weights
Create left weights
Create right weights
Create out weights


In [4]:


print("Create down processes")
ff_down = Sparse(weights=ff_weights)
rec_down = Sparse(weights=rec_weights_down)
lif_down = LIF(shape=frame_input.shape,
              du=du,
              dv=dv,
              vth=vth)

print("Create up processes")
ff_up = Sparse(weights=ff_weights)
rec_up = Sparse(weights=rec_weights_up)
lif_up = LIF(shape=frame_input.shape,
              du=du,
              dv=dv,
              vth=vth)

print("Create left processes")
ff_left = Sparse(weights=ff_weights)
rec_left = Sparse(weights=rec_weights_left)
lif_left = LIF(shape=frame_input.shape,
              du=du,
              dv=dv,
              vth=vth)

print("Create right processes")
ff_right = Sparse(weights=ff_weights)
rec_right = Sparse(weights=rec_weights_right)
lif_right = LIF(shape=frame_input.shape,
                du=du,
                dv=dv,
                vth=vth)

print("Create out processes")

sparse_out_up = Sparse(weights=w_out)
out_lif_up = LIF(shape=(num_out_neurons, ),
                 du=du,
                 dv=dv,
                 vth=vth)

sparse_out_down = Sparse(weights=w_out)
out_lif_down = LIF(shape=(num_out_neurons, ),
                 du=du,
                 dv=dv,
                 vth=vth)

sparse_out_left = Sparse(weights=w_out)
out_lif_left = LIF(shape=(num_out_neurons, ),
                 du=du,
                 dv=dv,
                 vth=vth)

sparse_out_right = Sparse(weights=w_out)
out_lif_right = LIF(shape=(num_out_neurons, ),
                 du=du,
                 dv=dv,
                 vth=vth)

recv = VisSwipeProcess(shape=frame_input.s_out.shape,
                       direction_shape=(num_out_neurons, ))



Create down processes
Create up processes
Create left processes
Create right processes
Create out processes


In [5]:
print("Connect")
# Connect
frame_input.s_out.connect(recv.frame_in)

frame_input.s_out.flatten().connect(ff_down.s_in)
ff_down.a_out.reshape(lif_down.a_in.shape).connect(lif_down.a_in)
lif_down.s_out.flatten().connect(rec_down.s_in)
rec_down.a_out.reshape(lif_down.a_in.shape).connect(lif_down.a_in)
lif_down.s_out.flatten().connect(sparse_out_down.s_in)
sparse_out_down.a_out.connect(out_lif_down.a_in)
out_lif_down.s_out.connect(recv.down_in)
# lif_down.s_out.connect(recv.frame_in)

frame_input.s_out.flatten().connect(ff_up.s_in)
ff_up.a_out.reshape(lif_up.a_in.shape).connect(lif_up.a_in)
lif_up.s_out.flatten().connect(rec_up.s_in)
rec_up.a_out.reshape(lif_up.a_in.shape).connect(lif_up.a_in)
lif_up.s_out.flatten().connect(sparse_out_up.s_in)
sparse_out_up.a_out.connect(out_lif_up.a_in)
out_lif_up.s_out.connect(recv.up_in)
# lif_up.s_out.connect(recv.frame_in)

frame_input.s_out.flatten().connect(ff_left.s_in)
ff_left.a_out.reshape(lif_left.a_in.shape).connect(lif_left.a_in)
lif_left.s_out.flatten().connect(rec_left.s_in)
rec_left.a_out.reshape(lif_left.a_in.shape).connect(lif_left.a_in)
lif_left.s_out.flatten().connect(sparse_out_left.s_in)
sparse_out_left.a_out.connect(out_lif_left.a_in)
out_lif_left.s_out.connect(recv.left_in)
# lif_left.s_out.connect(recv.frame_in)

frame_input.s_out.flatten().connect(ff_right.s_in)
ff_right.a_out.reshape(lif_right.a_in.shape).connect(lif_right.a_in)
lif_right.s_out.flatten().connect(rec_right.s_in)
rec_right.a_out.reshape(lif_right.a_in.shape).connect(lif_right.a_in)
lif_right.s_out.flatten().connect(sparse_out_right.s_in)
sparse_out_right.a_out.connect(out_lif_right.a_in)
out_lif_right.s_out.connect(recv.right_in)
# lif_right.s_out.connect(recv.frame_in)



Connect



<table>
<tr>
    <td> <img src="gifs/swipe.gif" alt="Drawing" style="height: 250px;"/> </td>
</tr>
</table>

In [6]:
print("Run")
# Run
num_steps = 10000
run_cfg = Loihi2SimCfg(exception_proc_model_map={Dense: PyDenseModelBitAcc, Sparse: PySparseModelBitAcc, LIF: PyLifModelBitAcc})
run_cnd = RunSteps(num_steps=num_steps)

frame_input.run(condition=run_cnd, run_cfg=run_cfg)
frame_input.stop()

Run


Process SystemProcess-2:
Process SystemProcess-23:
Process SystemProcess-7:
Process SystemProcess-15:
Process SystemProcess-22:
Process SystemProcess-17:
Process SystemProcess-9:
Process SystemProcess-18:
Process SystemProcess-19:
Process SystemProcess-16:
Process SystemProcess-13:
Process SystemProcess-10:
Process SystemProcess-4:
Process SystemProcess-20:
Process SystemProcess-24:
Process SystemProcess-21:
Process SystemProcess-5:
Process SystemProcess-14:
Process SystemProcess-8:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process SystemProcess-3:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process SystemProcess-6:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceba

KeyboardInterrupt: 